In [ ]:
from requests import get
from bs4 import BeautifulSoup
import re
import string

In [ ]:
def retrieve_data(question_list, answer_list, url, heading_list):
    """function retrieves questions and answers from the RUG FAQ except Immigration"""
    prev_question = ""
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    data = html_soup.findAll('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
    main_page = html_soup.findAll('div',{'class':'rug-list--accordion__item__header js--togglable-switch'})
    
    for links, headers in zip(data, main_page):
        link = links.findAll('a')
        header = headers.text
        header = header.strip()
        
        if header in heading_list: #exceptions for some pages
            question = header
            question = question_processing(question)   
            answer_links = link
            final_links = link_processing(answer_links)
            answer = clean_answer(links.text)
            answer_list.append([answer,final_links])
            question_list.append(question)
        
        else:
            for a in link:
                if a['href'][0] == "?": #check if link is an internal one
                    current_url = 'https://www.rug.nl/education/faq/'+a['href']          
                    response = get(current_url)
                    html_soup = BeautifulSoup(response.text, 'html.parser')
                    questions = html_soup.find_all(['h2'])
                    content = html_soup.findAll('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                    s_content = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                    overzicht_url.append(current_url)
                    
                    if len(content) == len(questions): #check structure of current questions and answers
                        for question, answer in zip(questions, content):
                            if question.text in heading_list: #check if question contains sub questions
                                links = answer.find_all('a')

                                for a in links: # access the subquestions
                                    question = a.text
                                    question = question_processing(question)
                                    
                                    if a['href'][0] == "?":
                                        current_url = 'https://www.rug.nl/education/faq/'+a['href']
                                        overzicht_url.append(current_url)
                                        response = get(current_url)
                                        html_soup = BeautifulSoup(response.text, 'html.parser')
                                        ans = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                                    
                                    elif a['href'][0] == "/":
                                        current_url = 'https://www.rug.nl'+a['href']
                                        overzicht_url.append(current_url)
                                        response = get(current_url)
                                        html_soup = BeautifulSoup(response.text, 'html.parser')
                                        ans = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                                    
                                    else: 
                                        current_url = a['href']
                                        overzicht_url.append(current_url)
                                        response = get(current_url)
                                        html_soup = BeautifulSoup(response.text, 'html.parser')
                                        ans = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})

                                    try: #catch errors when external page is formatted differently
                                        answer_links = ans.find_all('a')
                                        final_links = link_processing(answer_links)
                                        answer = clean_answer(ans.text)
                                        answer_list.append([answer,final_links])
                                        question_list.append(question)

                                    except(AttributeError, KeyError) as error: #Structure of external url is different
                                        ans = html_soup.find(['p'])
                                        answer_links = ans.find_all('a')
                                        final_links = link_processing(answer_links)
                                        answer = clean_answer(ans.text)
                                        answer_list.append([answer,final_links])
                                        question_list.append(question)

                            else:
                                question = question_processing(question.text)
                                answer_links = answer.find_all('a')
                                final_links = link_processing(answer_links)
                                answer = clean_answer(answer.text)
                                answer_list.append([answer,final_links])
                                question_list.append(question)

                    else:
                        question = html_soup.find(['h1'])
                        question = question_processing(question.text)
                        if prev_question == question: #no duplicate questions
                            pass
                        
                        else:
                            answer_links = s_content.find_all('a')
                            final_links = link_processing(answer_links)
                            answer = clean_answer(s_content.text)
                            question_list.append(question)
                            answer_list.append([answer,final_links])
                            prev_question = question

                elif a['href'][0] == '/':
                    current_url = 'https://www.rug.nl'+a['href']
                    overzicht_url.append(current_url)
                    response = get(current_url)
                    html_soup = BeautifulSoup(response.text, 'html.parser')
                    ans = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})
                    question = a.text
                    question = question_processing(question)

                    try:               
                        answer_links = ans.find_all('a')
                        final_links = link_processing(answer_links)
                        answer = clean_answer(ans.text)
                        answer_list.append([answer,final_links])
                        question_list.append(question)

                    except(AttributeError, KeyError) as error: #Structure of external url is different
                        ans = html_soup.find(['p'])
                        answer_links = ans.find_all('a')
                        final_links = link_processing(answer_links)
                        answer = clean_answer(ans.text)
                        answer_list.append([answer,final_links])
                        question_list.append(question)

                elif a['href'][0] == 'h':
                    question = a.text
                    question = question_processing(question)
                    final_links = []
                    current_url = a['href']
                    overzicht_url.append(current_url)
                    response = get(current_url)
                    html_soup = BeautifulSoup(response.text, 'html.parser')
                    ans = html_soup.find('div',{'class':'rug-clearfix rug-theme--content rug-mb'})

                    if ans == None:
                        answer = "Alleen een link gevonden"
                        final_links.append(a['href'])
                        answer_list.append([answer,final_links])
                        question_list.append(question)
                    
                    else:
                        answer_links = ans.find_all('a')
                        final_links = link_processing(answer_links)
                        answer = clean_answer(ans.text)                     
                        answer_list.append([answer,final_links])
                        question_list.append(question)

                else:
                    question = headers.text
                    answer_links = links.find_all('a')
                    final_links = link_processing(answer_links)
                    answer = clean_answer(links.text)
                    question = question_processing(question)
                    question_list.append(question)
                    answer_list.append([answer,final_links])
        
    return question_list, answer_list


In [ ]:
def clean_links(link_list):
    '''function that cleans links depending on their target location'''
    clean_links = []
    for link in link_list:
        if link == None: # no link in answer
            clean_links.append(None)
            pass
        
        else:
            link = link['href']
            if link[0] == '?': # target is sub page of the faq
                f_link = 'https://www.rug.nl/education/faq/' + link
                clean_links.append(f_link)
            
            elif link[0] == '/': # target is sub page of the rug site
                f_link = 'https://www.rug.nl' + link
                clean_links.append(f_link)
            
            else: # target is a different link
                clean_links.append(link)

    return clean_links

In [ ]:
def clean_answer(input_text):
    '''function that cleans the answer texts'''
    
    end = "Stel je vraag"
    answer = input_text.strip('\n')
    answer = answer.replace('\n',' ')
    answer = answer.replace('&nbsp;', ' ')
    answer = answer.replace('\xa0', ' ')
    honours = "Heb je nog vragen? Neem contact op met het Honours College via honours rug.nl."
    answer = answer.replace(honours, '')
    
    if answer.endswith(end):
        answer = answer.replace(end,'')
    return answer  

In [ ]:
def link_processing(answer_links):
    '''function that processes zero or more links in an answer'''
    for link in answer_links:
        if link.text == 'Stel je vraag':
            answer_links.remove(link)
    final_links = clean_links(answer_links)
    return final_links

In [ ]:
def question_processing(question):
    '''function that removes question marks from questions'''
    question = question.replace('?', '')
    return question

In [ ]:
def button_creation(link):
    '''function that creates aiml buttons for links'''
    
    button = '\n'+ '<button>' + '\n' + '<text>' + '\n'+ 'Klik hier' + '\n' + '</text>' + '\n' + '<url>'+ '\n' +link+ '\n' + '</url>' + '\n' + '</button>'
    return button

In [ ]:
def write_aiml_file(question_list, answer_list):
    '''function that automatically generates aiml file from questions, answers and links in answers'''
    seen = []
    with open('aiml_base.aiml', 'w') as f:
        for vraag, antwoord in zip(question_list, answer_list):
            buttons = ""
            if vraag in seen:
                pass
            else:
                if len(antwoord[1]) > 0: #answer contains one or more links; buttons have to be created
                    for link in antwoord[1]:
                        buttons += button_creation(link)

                    f.write('<category>'+'\n'+ '<pattern>'+ '\n'+ vraag+ '\n'+ '</pattern>'+'\n'+
                    '<template>'+ '\n'+ antwoord[0] + buttons + '\n' '</template>'+'\n'+'</category>'+'\n'+'\n')
                    seen.append(vraag)

                else:
                    f.write('<category>'+'\n'+ '<pattern>'+ '\n'+ vraag+ '\n'+ '</pattern>'+'\n'+
                    '<template>'+ '\n'+ antwoord[0]+ '\n'+ '</template>'+'\n'+ '</category>'+'\n' + '\n')
                    seen.append(vraag)
    f.close()

In [ ]:
# all pages from the FAQ except immigration
url_list = ['https://www.rug.nl/education/faq/?tcid=verint_3_7394_7394',
            'https://www.rug.nl/education/faq/?tcid=verint_3_7395_7395',
            'https://www.rug.nl/education/faq/?tcid=verint_3_7412_7412', 
            'https://www.rug.nl/education/faq/?tcid=verint_3_7398_7398', 
            'https://www.rug.nl/education/faq/?tcid=verint_3_7399_7399',
            'https://www.rug.nl/education/faq/?tcid=verint_3_7400_7400',
            'https://www.rug.nl/education/faq/?tcid=verint_3_7402_7402', 
            'https://www.rug.nl/education/faq/?tcid=verint_3_7401_7401']


#complete list of headings for exceptions in the site structure
heading_list = ["Introductieperiode", "Vakken, tentamens, boeken", "Aanmelding- en selectieprocedure", 
                "Onderwijs - honoursprogramma", "Ik sta voor meerdere opleidingen ingeschreven", 
                "Ik schrijf me tijdens het studiejaar in", "Aanmelding- en selectieprocedure", 
                "Ik heb nog geen Studielinkaccount","Ik heb al een Studielinkaccount","Onderwijs - honoursprogramma",
                "Aanmaken RUG-account", "Blokkeren RUG-account", "Opheffen RUG-account", "Veiligheid en toezicht tijdens de introductieperiode",
                "Onderwijs- en examenregelingen (OER)","Studentenstatuut","Klachten, bezwaar en beroep", 
                "Vertrouwenspersoon en studentendecaan", "Meer informatie rechtspositie ...", 
                "Aanvragen verklaring (door organisatie)","Career Services", "Aanmelden numerus fixusopleiding", 
                "Selectiepogingen", "Uitslag en bewijs van toelating", "VWO vooropleiding","HBO vooropleiding (bachelor en propedeuse)", 
                "Buitenlandse vooropleiding", "Andere vooropleiding", "Universitaire bachelor", "Introductieperiode", "Vakken, tentamens, boeken", "Collegegeld in termijnen betalen"]




In [ ]:
question_list = []
answer_list = []
overzicht_url = []

for url in url_list:
    question_list, answer_list, = retrieve_data(question_list, answer_list, url, heading_list)

print(len(question_list),len(answer_list))

In [ ]:
print(len(overzicht_url))
print(overzicht_url)

In [ ]:
write_aiml_file(question_list, answer_list)

In [ ]:
#remove aiml unfriendly tags
questions = []
answers = []
for vraag, antwoord in zip(question_list, answer_list):
    vraag = re.sub("<|>|/|", "", vraag)
    antwoord_text = re.sub("<|>|/|", "", antwoord[0])
    vraag = vraag.replace("&", "&amp;")
    antwoord_text = antwoord_text.replace("&", "&amp;")
    antwoord_text = antwoord_text.translate(str.maketrans('', '', string.punctuation))
    questions.append(vraag)
    answers.append([antwoord_text,antwoord[1]])
write_aiml_file(questions,answers)

In [ ]:
#process questions
with open('vragen.txt', 'w') as file:   
    for vraag in questions:
        file.write(vraag+'\n')
file.close()

In [ ]:
vragen = open(u"vragen.txt")
vragen = vragen.read()
print(vragen)